# Viewing Trending Videos Database

In this notebook we take a look at the database we created, which contains several metrics on trending videos of YouTube.

The database is automatically updated every day by inserting 50 new rows, the maximum number of videos supplied in the YouTube API trending videos list. We use Amazon Lambda with Amazon EventBridge Scheduler to invoke a python function every day. The Lambda function utilizes YouTube API to get the trending videos list, and retrieve several metrics of the videos in the list. The data is written in a PostgreSQL database in Amazon RDS.

The python script utilized in the Lambda function to collect data is contained as a separate file in this folder.

The goal of this project is to eventually analyze trending videos after collecting data for a few months.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from configparser import ConfigParser

In [2]:
# Load configuration from file
config = ConfigParser()
config.read('config.ini')

# PostgreSQL database configuration
host = config.get('database', 'host')
user = config.get('database', 'user')
password = config.get('database', 'password')
database = config.get('database', 'database')
port = 5432

In [3]:
# Construct the database URL
db_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create a SQLAlchemy engine
engine = create_engine(db_url)

query = "SELECT * FROM youtube_metrics"
df = pd.read_sql_query(query, engine)

df

,video_id,trending_date,publish_date,title,duration,views,likes,comments
0,Wjij-OX9RKI,2023-12-03,2023-12-02,Fortnite Battle Royale Chapter 5 Launch Trailer,0 days 00:01:30,4874502,230120,16151
1,F9U-yoJbgWs,2023-12-03,2023-12-02,The Boys – Season 4 Official Teaser Trailer | ...,0 days 00:01:50,5944265,314273,11029
2,0kQ8i2FpRDk,2023-12-03,2023-12-02,Fallout - Teaser Trailer | Prime Video,0 days 00:02:34,4738672,140347,12927
3,HQ8H5gqGA34,2023-12-03,2023-12-02,House of the Dragon Season 2 | Official Teaser...,0 days 00:01:26,3250473,130514,4834
4,okvG6H0UaIs,2023-12-03,2023-12-02,Top 10 GIFS of ALL TIME,0 days 00:26:24,2063101,83318,3086
...,...,...,...,...,...,...,...,...
195,gLYCjQpxzF4,2023-12-06,2023-12-03,Manchester City v. Tottenham Hotspur | PREMIER...,0 days 00:14:07,1308332,16940,1701
196,F9U-yoJbgWs,2023-12-06,2023-12-02,The Boys – Season 4 Official Teaser Trailer | ...,0 days 00:01:50,9068898,407665,13861
197,g_x0LVnITTg,2023-12-06,2023-12-04,Firefight: King of the Hill Trailer | Season 5...,0 days 00:01:12,186233,13165,2044
198,HQ8H5gqGA34,2023-12-06,2023-12-02,House of the Dragon Season 2 | Official Teaser...,0 days 00:01:26,6016204,182199,6837


In [4]:
query = """SELECT trending_date td, COUNT(*) entry_count
        FROM youtube_metrics
        GROUP BY td
        ORDER BY td
        """
pd.read_sql_query(query, engine)

,td,entry_count
0,2023-12-03,50
1,2023-12-04,50
2,2023-12-05,50
3,2023-12-06,50


This dataframe contains the current status of our database after four days of inserting new data.

Note that both `video_id` and `trending_date` are primary keys, so if a video appears in the trending videos list for more than a day, that is also recorded.